In [15]:
import requests
import pandas as pd
import re

def extract_album_id(album_link):
    match = re.search(r'/album/.*?/(\w+)\.html', album_link or "")
    return match.group(1) if match else "N/A"

def determine_album_type(track_count):
    if track_count == 1:
        return "Single"
    elif 2 <= track_count <= 6:
        return "EP"
    else:
        return "Regular"

def fetch_artist_songs(artist_name):
    url = f"http://localhost:5000/api/artistsongs?name={artist_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        songs = data.get("songs", [])
        
        records = []
        album_tracks = {}
        
        for song in songs:
            tracklist = song.get("tracklist", [])
            
           
            if not tracklist:
                tracklist = [{
                    "title": song.get("title", "N/A"),
                    "link": song.get("link", "N/A") 
                }]
            
            album_id = extract_album_id(song.get("albumLink", "") or "")
            
            if album_id not in album_tracks:
                album_tracks[album_id] = len(tracklist)
            
            for track in tracklist:
                track_link = track.get("link")  # Link bài hát từ tracklist
                
                # Nếu không có link trong tracklist, dùng link gốc từ dữ liệu bài hát
                if not track_link:
                    track_link = song.get("link", "N/A")

                records.append({
                    "album_id": album_id,
                    "album_name": song.get("album", song.get("title", "N/A")),
                    "track_title": track["title"],
                    "release_date": song.get("releaseDate", "Unknown"),
                    "provided_by": song.get("providedBy", "Unknown"),
                    "featured_artists": song.get("featuredArtists", "Unknown"),
                    "album_owner": song.get("albumOwner", "Unknown"),
                    "linkzingmp3": track_link  
                })
        
        df = pd.DataFrame(records)
        df = df.drop_duplicates(subset=["album_id", "album_name", "track_title"])
        
  
        df["album_type"] = df["album_id"].map(lambda x: determine_album_type(album_tracks.get(x, 1)))
        
        return df
    else:
        print("Error fetching data:", response.status_code)
        return None

artist_name = "Binh-Gold"

df = fetch_artist_songs(artist_name)
df.to_excel(f"{artist_name}_songZingMp3.xlsx", index=False)
df




,album_id,album_name,track_title,release_date,provided_by,featured_artists,album_owner,linkzingmp3,album_type
0,6CZFOUCE,ĐỔI TƯ THẾ (Single),ĐỔI TƯ THẾ,02/10/2024,MIXUS,"Bình Gold, Andree Right Hand","Bình Gold, Andree Right Hand",https://zingmp3.vn/bai-hat/DOI-TU-THE-Binh-Gol...,Single
1,6B6W6EDA,Em iu (Single),Em iu,30/01/2022,Believe,"Andree Right Hand, Wxrdie, Bình Gold, 2Pillz","Andree Right Hand, Wxrdie, Bình Gold, 2Pillz",https://zingmp3.vn/bai-hat/Em-iu-Andree-Right-...,Single
2,ZAE7ZAB8,Lái Máy Bay (Single),Lái Máy Bay,2019,MIXUS,Bình Gold,Bình Gold,https://zingmp3.vn/bai-hat/Lai-May-Bay-Binh-Go...,Single
3,ZUAOBEUZ,Nợ (Single),Nợ,2019,MIXUS,Bình Gold,Bình Gold,https://zingmp3.vn/bai-hat/No-Binh-Gold/ZWACD0...,Single
4,676WEBDI,Tuổi Gì Mà Chẳng Thích Lì Xì (feat. Bình Gold)...,Tuổi Gì Mà Chẳng Thích Lì Xì,16/01/2020,Warner Music Vietnam,"Bích Phương, Bình Gold","Bích Phương, Bình Gold",https://zingmp3.vn/bai-hat/Tuoi-Gi-Ma-Chang-Th...,Single
5,6BZWUBO6,Em iu (Max Benderz Remix) (Single),Em iu (Max Benderz Remix),30/01/2022,Believe,"Andree Right Hand, Wxrdie, Bình Gold, 2Pillz","Andree Right Hand, Wxrdie, Bình Gold, 2Pillz",https://zingmp3.vn/bai-hat/Em-iu-Max-Benderz-R...,Single
6,N/A,"Em Iu (Dustee, Monotape & Teddy Doox Remix)","Em Iu (Dustee, Monotape & Teddy Doox Remix)",None,None,"Wxrdie, Andree Right Hand, Bình Gold",None,https://zingmp3.vn/bai-hat/Em-Iu-Dustee-Monota...,Single
7,6C8BOO70,Quan Hệ Rộng (Remix) (Single),Quan Hệ Rộng (Remix),17/03/2025,VIEENT Music,"Bình Gold, KCV In Da House","Bình Gold, KCV In Da House",https://zingmp3.vn/bai-hat/Quan-He-Rong-Remix-...,Single
8,6C8BOOOU,Quan Hệ Rộng (Vip Mix) (Single),Quan Hệ Rộng (Vip Mix),17/03/2025,VIEENT Music,"Bình Gold, KCV In Da House","Bình Gold, KCV In Da House",https://zingmp3.vn/bai-hat/Quan-He-Rong-Vip-Mi...,Single
9,6B7FWBIF,Em iu (Single),Em iu,09/01/2023,Believe,"Wind.P, Andree Right Hand, Wxrdie, Bình Gold","Wind.P, Andree Right Hand, Wxrdie, Bình Gold",https://zingmp3.vn/bai-hat/Em-iu-Wind-P-Andree...,Single


In [3]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

def get_artist_tracks_all(artist_name):
    client_id = "c7e1fe3ffe674920a01f9b016e6ae5df"
    client_secret = "215c6808dea74656b3629b306182ac4b"
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
    
    # Tìm kiếm nghệ sĩ
    results = sp.search(q=artist_name, type='artist', limit=1)
    if not results['artists']['items']:
        print("Không tìm thấy nghệ sĩ.")
        return pd.DataFrame()
    
    artist_id = results['artists']['items'][0]['id']

    track_data = []
    query = f"artist:{artist_name}"
    search_results = sp.search(q=query, type='track', limit=50)  
    
    for track in search_results['tracks']['items']:
        track_title = track['name']
        link_spotify = track['external_urls']['spotify']
        album = track['album']
        album_name = album['name']
        album_release_date = album.get('release_date', 'Unknown')
        album_owner = album['artists'][0]['name']
        featured_artists = [artist['name'] for artist in track.get('artists', []) if artist['id'] != artist_id]
        featured_artists = ", ".join(featured_artists) if featured_artists else "None"

        
        # Lấy thông tin nhà cung cấp (label)
        album_id = album['id']
        album_info = sp.album(album_id)
        label = album_info.get('label', 'Unknown')

        try:
            album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
        except:
            album_release_date = "Unknown"
        
        track_data.append([
            album_name, track_title, album_release_date, 
            featured_artists if featured_artists else "None", 
            album_owner, label, link_spotify
        ])
    
    # Tạo DataFrame
    columns = ["album_name", "track_title", "release_date", "featured_artists", "album_owner", "label", "Link_Spotify"]
    df = pd.DataFrame(track_data, columns=columns)
    return df

if __name__ == "__main__":
    artist_name = 'Obito'
    df_tracks = get_artist_tracks_all(artist_name)


In [4]:
df_tracks

,album_name,track_title,release_date,featured_artists,album_owner,label,Link_Spotify
0,Lặng,1000 Ánh Mắt,27/06/2024,Shiki,Shiki,12 trái lê,https://open.spotify.com/track/5XfGQZA0ioQAWUj...
1,ii-V-I,Track 06,05/12/2024,"Tyronee, VSTRA",Tyronee,12 trái lê,https://open.spotify.com/track/72NMPuJCRsOUoXQ...
2,Đánh Đổi,Hà Nội,10/10/2023,"Shiki, VSTRA",Obito,12 trái lê,https://open.spotify.com/track/5PGh9MyKry8slG5...
3,Đánh Đổi,Đánh Đổi,10/10/2023,"Shiki, RPT MCK",Obito,12 trái lê,https://open.spotify.com/track/1UQNU5O9VttUgO1...
4,"Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...","Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...",25/12/2023,"VSOUL, RPT MCK, Ronboogz, Boyzed",VSOUL,12 trái lê,https://open.spotify.com/track/4OXAneZHFJyQV1R...
5,Đánh Đổi,Tell the kids i love them,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/2Asy51yFMEG8Qjh...
6,Đánh Đổi,Đầu Đường Xó Chợ,10/10/2023,"Shiki, Lăng LD",Obito,12 trái lê,https://open.spotify.com/track/2G5ujw3rrWcAXQq...
7,Đánh Đổi,Biên Giới Long Bình,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/5KXRShxe6Y54ueB...
8,Simple Love,Simple Love,28/11/2022,Seachains,Obito,12 trái lê,https://open.spotify.com/track/1hFZdk0jIkrMEri...
9,Đánh Đổi,Sài Gòn ơi,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/2OucKhBwbCuIdCq...


In [5]:
import re
import unicodedata

def normalize_title(title):
    """
    Chuẩn hóa tiêu đề bài hát: 
    - Chuyển thành chữ thường
    - Loại bỏ dấu tiếng Việt
    - Loại bỏ ký tự đặc biệt
    """
    # Chuyển thành chữ thường
    title = title.lower()
    
    # Loại bỏ dấu tiếng Việt
    title = ''.join(
        c for c in unicodedata.normalize('NFD', title)
        if unicodedata.category(c) != 'Mn'
    )

    # Loại bỏ ký tự đặc biệt (giữ lại chữ cái, số và khoảng trắng)
    title = re.sub(r"[^a-z0-9\s]", "", title)

    # Xóa khoảng trắng dư thừa
    title = re.sub(r"\s+", " ", title).strip()

    return title

df_tracks["normalized_title"] = df_tracks["track_title"].apply(normalize_title)
if "track_title" in df.columns:
    df["normalized_title"] = df["track_title"].apply(normalize_title)


# Gộp dữ liệu từ hai nguồn dựa trên cột "normalized_title"
df_merged = pd.merge(df_tracks, df, on="normalized_title", how="outer", suffixes=("_Spotify", "_ZingMP3"))
df_ = df_merged.drop(columns=["normalized_title",'album_name_ZingMP3','track_title_ZingMP3'])
df_ = df_.rename(columns={"album_name_Spotify":"album_name","track_title_Spotify": "tracklist(danh sách bài hát)","featured_artists_Spotify":"Song artist(nghệ sĩ tham gia bài hát)(Spotify)","album_owner_Spotify": "Album artist (nghệ sĩ sở hữu album)*(Spotify)","	release_date_Spotify":"Ngày phát hành trên Spotify","provided_by":"Cung cấp bởi(ZingMP3)",'label':'Cung cấp bởi(Spotify)',
                          "featured_artists_ZingMP3":"Song artist(nghệ sĩ tham gia bài hát)(ZingMP3)","album_owner_ZingMP3":"Album artist (nghệ sĩ sở hữu album)*(ZingMP3)","release_date_ZingMP3":"Ngày phát hành trên ZingMP3","linkzingmp3":"ZingMP3","Link_Spotify":"Spotify","album_id":"Mã định danh album zingmp3"})
# Hiển thị một số dòng đầu tiên
desired_columns = [
# #  "album_id","album_name","Album artist (nghệ sĩ sở hữu album)*","album_type","tracklist(danh sách bài hát)","Ngày phát hành trên ZingMP3","Cung cấp bởi","Song artist(nghệ sĩ tham gia bài hát)","Ngày phát hành trên Spotify", "Song artist(nghệ sĩ tham gia bài hát)","Album artist (nghệ sĩ sở hữu album)*","Cung cấp bởi","Mã định danh album ZingMP3","Link_ZingMP3","Link_Spotify"]
# # merged_df = merged_df[desired_columns]
	"album_name","Album artist (nghệ sĩ sở hữu album)*(Spotify)","Album artist (nghệ sĩ sở hữu album)*(ZingMP3)", "album_type",	"tracklist(danh sách bài hát)","release_date_Spotify","Ngày phát hành trên ZingMP3","Song artist(nghệ sĩ tham gia bài hát)(Spotify)","Song artist(nghệ sĩ tham gia bài hát)(ZingMP3)","Cung cấp bởi(ZingMP3)","Cung cấp bởi(Spotify)","Mã định danh album zingmp3","ZingMP3","Spotify"	
]
# Lưu kết quả vào file CSV (nếu cần)
#df_merged.to_csv("merged_artist_tracks.csv", index=False)
df_ = df_[desired_columns]
df_


,album_name,Album artist (nghệ sĩ sở hữu album)*(Spotify),Album artist (nghệ sĩ sở hữu album)*(ZingMP3),album_type,tracklist(danh sách bài hát),release_date_Spotify,Ngày phát hành trên ZingMP3,Song artist(nghệ sĩ tham gia bài hát)(Spotify),Song artist(nghệ sĩ tham gia bài hát)(ZingMP3),Cung cấp bởi(ZingMP3),Cung cấp bởi(Spotify),Mã định danh album zingmp3,ZingMP3,Spotify
0,Lặng,Shiki,Shiki,EP,1000 Ánh Mắt,27/06/2024,27/06/2024,Shiki,"Shiki, Obito",Warner Music Vietnam,12 trái lê,6CZ7U60W,https://zingmp3.vn/bai-hat/1000-Anh-Mat-Shiki-...,https://open.spotify.com/track/5XfGQZA0ioQAWUj...
1,Đánh Đổi,Obito,"Obito, Shiki",Regular,16,10/10/2023,10/10/2023,Shiki,"Obito, Shiki, Lăng LD",Warner Music Vietnam,12 trái lê,6BDOI9WU,https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,https://open.spotify.com/track/3jwzesnDs0jY7Ke...
2,Đánh Đổi,Obito,"Obito, Shiki",Regular,Đánh Đổi,10/10/2023,10/10/2023,"Shiki, RPT MCK","Obito, Shiki, Lăng LD",Warner Music Vietnam,12 trái lê,6BDOI9WU,https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,https://open.spotify.com/track/1UQNU5O9VttUgO1...
3,Đánh Đổi,Obito,"Obito, Shiki",Regular,Đầu Đường Xó Chợ,10/10/2023,10/10/2023,"Shiki, Lăng LD","Obito, Shiki, Lăng LD",Warner Music Vietnam,12 trái lê,6BDOI9WU,https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,https://open.spotify.com/track/2G5ujw3rrWcAXQq...
4,Đánh Đổi,Obito,"Obito, Shiki",Regular,Backstage Freestyle,10/10/2023,10/10/2023,Shiki,"Obito, Shiki, Lăng LD",Warner Music Vietnam,12 trái lê,6BDOI9WU,https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,https://open.spotify.com/track/6Kvm6QRtOOM6bMe...
5,Đánh Đổi,Obito,"Obito, Shiki",Regular,Biên Giới Long Bình,10/10/2023,10/10/2023,Shiki,"Obito, Shiki, Lăng LD",Warner Music Vietnam,12 trái lê,6BDOI9WU,https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,https://open.spotify.com/track/5KXRShxe6Y54ueB...
6,NaN,NaN,None,Single,NaN,NaN,None,NaN,"Lope Phạm, Obito",None,NaN,N/A,https://zingmp3.vn/bai-hat/Boom-Lope-Pham-Obit...,NaN
7,"Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...",VSOUL,"VSOUL, MCK, Obito, Ronboogz, Boyzed",Single,"Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...",25/12/2023,25/12/2023,"VSOUL, RPT MCK, Ronboogz, Boyzed","VSOUL, MCK, Obito, Ronboogz, Boyzed",Warner Music Vietnam,12 trái lê,6BEEZ96E,https://zingmp3.vn/bai-hat/Buon-Hay-Vui-feat-R...,https://open.spotify.com/track/4OXAneZHFJyQV1R...
8,OMERTÀ,$A Milo,"$A Milo, Gxxfy",Single,BUSINESS MODE,18/05/2024,17/05/2024,"$A Milo, B-Wine, Gxxfy","$A Milo, Gxxfy, Obito, B-Wine",DAO Music Entertainment,MaiDao Music,SC0EC0B6,https://zingmp3.vn/bai-hat/BUSINESS-MODE-A-Mil...,https://open.spotify.com/track/1nLkB5tZaxAIbJ7...
9,Can We Love,Obito,Obito,Single,Can We Love,24/03/2022,24/03/2022,None,Obito,Universal Music Group,Universal Music Indochina,6BW0ICWD,https://zingmp3.vn/bai-hat/Can-We-Love-Obito/Z...,https://open.spotify.com/track/7iXHkRf9Z4x6ona...


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Cấu hình Selenium WebDriver
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Chạy ẩn trình duyệt
chrome_options.add_argument("--disable-popup-blocking")

# Khởi tạo trình duyệt
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Giả định df_ đã được tải với dữ liệu cần thiết
df_missing_links = df_[df_['ZingMP3'].isna()]

# Danh sách để lưu kết quả tìm được
updated_links = []

for index, row in df_missing_links.iterrows():
    tracklist = row['tracklist(danh sách bài hát)']
    artist = row['Song artist(nghệ sĩ tham gia bài hát)(Spotify)']
    
    if pd.notna(tracklist) and pd.notna(artist):
        search_query = f"{tracklist} {artist}"
        zing_search_url = f"https://zingmp3.vn/tim-kiem/tat-ca?q={search_query.replace(' ', '+')}"

        try:
            driver.get(zing_search_url)
            time.sleep(3)  # Chờ trang load

            # Xử lý pop-up nếu có
            try:
                pop_up_close_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "zmp-modal-close") or contains(@class, "close")]'))
                )
                pop_up_close_button.click()
                print("Pop-up ZingMP3 Plus closed.")
            except:
                print("No pop-up found.")

            # Lấy link bài hát đầu tiên từ kết quả tìm kiếm
            search_results = driver.find_elements(By.XPATH, '//a[contains(@class, "media-title")]')

            if search_results:
                zingmp3_link = search_results[0].get_attribute("href")
                print(f"Found link: {zingmp3_link} for {tracklist} - {artist}")
                updated_links.append((index, zingmp3_link))
            else:
                print(f"No result for {tracklist} - {artist}")
                updated_links.append((index, None))

        except Exception as e:
            print(f"Error searching for {tracklist} - {artist}: {e}")
            updated_links.append((index, None))

# Đóng trình duyệt
driver.quit()

# Cập nhật lại DataFrame
for idx, link in updated_links:
    df_.at[idx, 'ZingMP3'] = link




No result for NGÀY MAI LÀ TƯƠNG LAI (feat. Obito) - RAP VIỆT
No result for NGHE - NÓI (feat. Pjpo & Obito) - RAP VIỆT, Pjpo
No result for Plata Blanca - Jakush314, Verzet, Krash, Prodigo, Kxtrplsx
No result for RAP CHO ANH EM - OCEAN MOB, Willistic, wAvy, Xolitxo, Gill
No result for VẪN LÀ TRAI TỐT (feat. Obito) - RAP VIỆT
No result for When You Look at Me (feat. Seachains) - Seachains
